In [6]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv  

load_dotenv()

True

In [7]:
KEY = os.getenv("OPENAI_API_KEY", None)
if not KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

In [8]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.5)

In [10]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [11]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [66]:
TEMPLATE = """Text = {text}
You are an expert mcq generator , Given the above text, it is your job to create a quiz of {number} multiple choice questions.
Each question should have 4 options, one of which is correct. The correct answer should be indicated by the letter 'A', 'B', 'C', or 'D'.
The questions should be of topic {subject} in {tone} tone. Make sure questions are not repeated mMake sure to format your response like  RESPONSE_JSON below  and use it as a guide.

### RESPONSE_JSON
{response_json}
"""

In [67]:
quiz_generator_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE)

In [68]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generator_prompt, output_key="quiz", verbose=True)

In [69]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [70]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"],
                                        template=TEMPLATE)

In [71]:
quiz_evaluation_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="evaluation", verbose=True)


In [72]:
generate_evaluate_quiz_chain = SequentialChain(
    chains=[quiz_chain, quiz_evaluation_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "evaluation"],
    verbose=True
)   

In [73]:
with open("../data.txt", "r") as file:
    TEXT = file.read()

In [74]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [75]:
NUMBER = 5
SUBJECT = "Computer Science"
TONE = "Professional"

In [76]:
with get_openai_callback() as cb:
    response=generate_evaluate_quiz_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Text = Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defin

In [77]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [79]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [80]:
quiz=response.get("quiz")


In [81]:
quiz=json.loads(quiz)


In [82]:
quiz 

{'1': {'mcq': 'What is the main theme in biology that explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Energy processing',
   'c': 'Cellular structure',
   'd': 'Internal regulation'},
  'correct': 'a'},
 '2': {'mcq': 'What is a key characteristic of all organisms according to biology?',
  'options': {'a': 'Made up of cells',
   'b': 'Ability to fly',
   'c': 'Inorganic composition',
   'd': 'Lack of genetic information'},
  'correct': 'a'},
 '3': {'mcq': 'Which level of organization in biology focuses on the molecular biology of a cell?',
  'options': {'a': 'Cellular level',
   'b': 'Organismal level',
   'c': 'Population level',
   'd': 'Community level'},
  'correct': 'a'},
 '4': {'mcq': 'What scientific method do biologists use to make observations and form conclusions?',
  'options': {'a': 'Scientific method',
   'b': 'Trial and error',
   'c': 'Guesswork',
   'd': 'Intuition'},
  'correct': 'a'},
 '5': {'mcq': 'How long ago did life on Earth em

In [85]:
quiz_table = []

for key, value in quiz.items():
    question = value.get("mcq", "")
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct_answer = value.get("correct", "")
    
    quiz_table.append({"MCQ": question, "Choices": options, "Correct": correct_answer})


In [86]:
quiz_table

[{'MCQ': 'What is the main theme in biology that explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Energy processing | c: Cellular structure | d: Internal regulation',
  'Correct': 'a'},
 {'MCQ': 'What is a key characteristic of all organisms according to biology?',
  'Choices': 'a: Made up of cells | b: Ability to fly | c: Inorganic composition | d: Lack of genetic information',
  'Correct': 'a'},
 {'MCQ': 'Which level of organization in biology focuses on the molecular biology of a cell?',
  'Choices': 'a: Cellular level | b: Organismal level | c: Population level | d: Community level',
  'Correct': 'a'},
 {'MCQ': 'What scientific method do biologists use to make observations and form conclusions?',
  'Choices': 'a: Scientific method | b: Trial and error | c: Guesswork | d: Intuition',
  'Correct': 'a'},
 {'MCQ': 'How long ago did life on Earth emerge according to the text?',
  'Choices': 'a: More than 3.7 billion years ago | b: Less than 1 million years ago

In [87]:
quiz = pd.DataFrame(quiz_table)

In [88]:
quiz

,MCQ,Choices,Correct
0,What is the main theme in biology that explain...,a: Evolution | b: Energy processing | c: Cellu...,a
1,What is a key characteristic of all organisms ...,a: Made up of cells | b: Ability to fly | c: I...,a
2,Which level of organization in biology focuses...,a: Cellular level | b: Organismal level | c: P...,a
3,What scientific method do biologists use to ma...,a: Scientific method | b: Trial and error | c:...,a
4,How long ago did life on Earth emerge accordin...,a: More than 3.7 billion years ago | b: Less t...,a


In [89]:
quiz.to_csv("machinelearning.csv",index=False)
